In [8]:
from sqlalchemy import create_engine
from models import Base, Product

DATABASE_URL = "sqlite:///products.db"

engine = create_engine(DATABASE_URL, echo=True)
Base.metadata.create_all(bind=engine)

2023-10-04 13:18:34,528 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-04 13:18:34,540 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("products")
2023-10-04 13:18:34,541 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-04 13:18:34,542 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class Product(Base):
    __tablename__ = 'products'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    price = Column(Float)
    image_path = Column(String)


In [9]:
from sqlalchemy import create_engine, MetaData

# データベースへの接続
DATABASE_URL = "sqlite:///products.db"  # 適切なデータベースのURLを指定してください
engine = create_engine(DATABASE_URL)

# メタデータの取得
metadata = MetaData()
metadata.reflect(bind=engine)

# テーブルのリストを取得
tables = metadata.tables.keys()

tables

dict_keys(['products'])

In [15]:
from sqlalchemy import Table
table_name = "products"  # 確認したいテーブル名を指定します

# テーブルオブジェクトを取得
table = Table(table_name, metadata, autoload=True, autoload_with=engine)

# テーブルの内容を取得
results = engine.execute(table.select()).fetchall()

results

2023-10-04 13:20:32,507 INFO sqlalchemy.engine.Engine SELECT products.id, products.name, products.price, products.image_path 
FROM products
2023-10-04 13:20:32,508 INFO sqlalchemy.engine.Engine [generated in 0.00094s] ()


[(1, '教材1', 1000.0, 'kyozai1.jpg')]

In [12]:
from sqlalchemy.orm import sessionmaker
SessionLocal = sessionmaker(bind=engine)
def delete_product_by_id(product_id):
    session = SessionLocal()
    try:
        product = session.query(Product).filter_by(id=product_id).first()
        if product:
            session.delete(product)
            session.commit()
        else:
            print("Product not found!")
    except Exception as e:
        print(f"Error: {e}")
        session.rollback()
    finally:
        session.close()

# 使用例
delete_product_by_id(1)

In [17]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from models import Product

DATABASE_URL = "sqlite:///products.db"

engine = create_engine(DATABASE_URL, echo=True)
Session = sessionmaker(bind=engine)

# 新しい商品を追加
session = Session()
new_product = Product(name="教材3", price=1700.0, image_path="kyozai3.jpg")
session.add(new_product)
session.commit()

# 全商品の取得
products = session.query(Product).all()
for product in products:
    print(product.id, product.name, product.price, product.image_path)

session.close()

2023-10-04 13:21:04,085 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-04 13:21:04,087 INFO sqlalchemy.engine.Engine INSERT INTO products (name, price, image_path) VALUES (?, ?, ?)
2023-10-04 13:21:04,088 INFO sqlalchemy.engine.Engine [generated in 0.00067s] ('教材3', 1700.0, 'kyozai3.jpg')
2023-10-04 13:21:04,090 INFO sqlalchemy.engine.Engine COMMIT
2023-10-04 13:21:04,197 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-04 13:21:04,198 INFO sqlalchemy.engine.Engine SELECT products.id AS products_id, products.name AS products_name, products.price AS products_price, products.image_path AS products_image_path 
FROM products
2023-10-04 13:21:04,199 INFO sqlalchemy.engine.Engine [generated in 0.00086s] ()
1 教材1 1000.0 kyozai1.jpg
2 教材2 1500.0 kyozai2.jpg
3 教材3 1700.0 kyozai3.jpg
2023-10-04 13:21:04,201 INFO sqlalchemy.engine.Engine ROLLBACK
